In [1]:
import aitk.robots
from aitk.utils import Grid
aitk.robots.__version__

'0.9.43'

# Maze robot

One approach for solving a maze is called the *right-hand rule*.  You continually follow the wall on your right-hand side, and eventually, if there is an exit, you will find your way out.  We will be trying to implement this with a robot, therefore our robot will need to be able to sense walls on its right side as well as walls in front. The robot has been given two RangeSensors on its right side, one close to the front and the other close to the rear.  The robot has also been given one RangeSensor centered on its front.  The robot has been given one LightSensor so that it can detect when the goal has been found.

In [2]:
robot = aitk.robots.Scribbler(x=20, y=180, a=87, max_trace_length=60)
robot.add_device(aitk.robots.RangeSensor(position=(6,0),max=20,a=0,width=57.3,name="f-ir"))
robot.add_device(aitk.robots.RangeSensor(position=(6,5),max=30,a=270,width=57.3,name="rf-ir"))
robot.add_device(aitk.robots.RangeSensor(position=(-6,5),max=30,a=270,width=57.3,name="rb-ir"))
robot.add_device(aitk.robots.LightSensor(position=(6,0),name="light-sensor"))

# Maze world

The following code creates a maze where the exit is denoted by a bulb in the lower right corner of the world. The robot's goal is to reach this light source when starting from the lower left corner of the world.

In [3]:
maze_world = aitk.robots.World(width=400, height=200, scale=2.0)
maze_world.add_bulb("yellow", 380, 180, 0, 100)
maze_world.add_wall("blue", 45, 200, 50, 50)
maze_world.add_wall("blue", 95, 0, 100, 75)
maze_world.add_wall("blue", 95, 200, 100, 125)
maze_world.add_wall("blue", 145, 50, 150, 150)
maze_world.add_wall("blue", 145, 150, 350, 145)
maze_world.add_wall("blue", 145, 45, 350, 50)
maze_world.add_wall("blue", 200, 95, 400, 100)
#Comment out the wall below to make a shorter path to the exit
maze_world.add_wall("blue", 345, 150, 350, 400)

maze_world.add_robot(robot)
maze_world.update()
maze_world.save()

Random seed set to: 9026240


In [4]:
maze_world.watch()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

# Maze controller
Below is the start to a controller for traversing the maze.  You need to finish this. Your goal is to try to have the robot follow the wall on its right hand side until its LightSensor senses that the bulb is very close (the light reading should be greater than 0.9).  

For debugging purposes, it may be helpful to have the robot speak its RangeSensor readings.  This should make it easier to figure out how to ensure that the robot follows the wall, without getting too close or too far away from it.  

A good strategy is to add one or two new cases to the if-else statement at a time and test them. For example:
- You could start by adding a case to handle when the front RangeSensor is triggered.
- Next you could add a case to handle when the right RangeSensors are not sensing a wall at all. 
- You'll likely want to have cases to handle if either the nose or the rear of the robot gets too close to the wall.


In [38]:
def traverse_maze(robot):
    if robot.stalled:
        robot.speak("STUCK")
        return True
    #if light found
    elif robot["light-sensor"].get_brightness() > 0.9:
        robot.speak("FOUND LIGHT")
        return True
    elif robot["f-ir"].get_distance() < robot["f-ir"].get_max():
        robot.speak("Front close %.1f" % (robot["f-ir"].get_distance()))
        robot.move(-0.3,0)
        if robot["rf-ir"].get_distance() < 10 and robot["rb-ir"].get_distance() < 10 and robot["f-ir"].get_distance() < 10:
            robot.move(0.5,0.5)
        elif robot["rf-ir"].get_distance() > 10 and robot["rb-ir"].get_distance() > 10:
            robot.move(0,-0.3)
        else:
            robot.move(0.5,0.3)
    elif robot["rf-ir"].get_distance() > 20 and robot["rb-ir"].get_distance() > 20:
        robot.speak("Wall turn %.1f %.1f" % (robot["rf-ir"].get_distance(),robot["rb-ir"].get_distance()))
        robot.move(0.5,-0.3)
       
    else:
        robot.speak("Forward")
        robot.move(1.0, 0) 

In [39]:
maze_world.reset()
maze_world.run([traverse_maze])

Using random seed: 9026240


0it [00:00, ?it/s]

Simulation stopped at: 00:01:57.50; speed 0.99 x real time


True

# Reflect on the robot controller

In what ways is the `traverse_maze` controller **subsymbolic**? 

The controller is subsymbolic in that it has the robot react and respond to various stimuli and interact with its environment through its motion and sensors

In what ways is it **symbolic**?

The controller is symbolic in that it essentially comprises an algorithm for traversing the maze rather than having the robot learn and improve from its time in the maze

# Reflect on the robot environment

From the perspective of the robot, is this maze world
- fully or partially observable?
- deterministic or stochastic?
- static or dynamic?
- discrete or continuous?

Explain your reasoning behind each answer.

- The maze would be a partially observable environment as the robot does not have any knowledge of its layout prior to exploring it
- The maze is deterministic as the robot has a set of generalised instructions with which it approaches the maze
- The maze is static, as its layout and nature does not change over time
- The maze is discrete, as it does not change and there are a finite (though large) number of possible ways in which the robot can approach it